In [492]:
import numpy as np
import pandas as pd
import os
import re
import os.path
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [433]:
def get_session_data(ses_names):
    """
    Given a list of session names, opens all session folders and
    saves all txt files, country names and session years
    into three lists.
    """
    path = os.getcwd()  
    all_sessions = []
    all_countries = []
    years = []
    for session in ses_names:
        sessions = []
        countries = []
        f_path = os.path.join(path,session)
        years.append(int(f_path[-4:]))
        files = os.listdir(f_path)
        for file in files:
             if file.endswith('.txt'):
                with open(os.path.join(f_path,file), 'r', encoding='utf-8') as f:
                    countries.append(file.split('_')[0])
                    sessions.append(f.read())
        all_sessions.append(sessions)
        all_countries.append(countries)

    return all_sessions, all_countries, years 

In [522]:
# Get the names of all sessions in the current directory
ses_names = [filename for filename in os.listdir() if filename.startswith('Session')]
ses_names[:5]

['Session 29 - 1974',
 'Session 50 - 1995',
 'Session 64 - 2009',
 'Session 51 - 1996',
 'Session 27 - 1972']

In [434]:
all_sessions, all_countries, years = get_session_data(ses_names)

In [546]:
print('Total number of sessions is ', len(all_sessions))
print('Number of countries in General Debate in',years[0], 'is', len(all_sessions[0]))

Total number of sessions is  49
Number of countries in General Debate in 1974 is 129


# Sentiment Analysis 

In [543]:
def get_sentiment(text):
    """
    Given a text, computes polarity scores and returns 1 if
    the text's sentiment is negative and zero otherwise.
    A sentiment is negative if the compound score is lower than -0.05.
    """
    sid = SentimentIntensityAnalyzer()
    sentiment_dict = sid.polarity_scores(text)
    count = 0
    if sentiment_dict['compound'] <= - 0.05:
        count += 1
    return count

In [459]:
def get_neg_sentiments(all_sessions, all_countries, years): #list of lists
    all_neg_sents = []
    ses_len = len(all_sessions)

    for i in range(ses_len):
        neg_sentim_list = []
        for j in range(len(all_sessions[i])):
            if get_sentiment(all_sessions[i][j]) > 0:
                neg_sentim_list.append(all_countries[i][j])
        all_neg_sents.append((years[i],neg_sentim_list))
        
    return all_neg_sents

In [460]:
neg_sents = get_neg_sentiments(all_sessions, all_countries, years)

In [480]:
neg_sents = sorted(neg_sents, key = lambda x: x[0])

In [486]:
neg_per_year = [ ]
for tup in neg_sents:
    neg_per_year.append(len(tup[1]))
    

In [523]:
data = [go.Bar(
            x = list(range(1970, 2019)),
            y = neg_per_year
    )]

layout = go.Layout(
    title='<b>Number of Negative Speeches from Each Session</b>',
    xaxis=dict(
        title='Session Year'
    ),
    yaxis=dict(
        title='Number of Negative Speeches'
    ),
    
)

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='Negative Speeches.html')

'file:///Users/anushkocharyan/Desktop/UN_Sessions/Negative Speeches.html'

In [466]:
neg_sent_countries = [ ]
for tup in neg_sents:
    neg_sent_countries.extend(tup[1]) 

In [500]:
top_neg_countries = Counter(neg_sent_countries).most_common(10)
top_neg_countries

[('IRN', 14),
 ('CUB', 13),
 ('ISR', 12),
 ('SYR', 11),
 ('LBY', 10),
 ('COL', 7),
 ('SGP', 6),
 ('MYS', 6),
 ('IRQ', 6),
 ('SAU', 5)]

In [501]:
neg_speech = []
for c in top_neg_countries:
    neg_speech.append(c[1])

In [524]:
# generate a plot of countries with the highest number of negative speeches
data = [go.Bar(
            x = ['Iran', 'Cuba', 'Israel', 'Syria', 'Lybia',
                 'Colombia', 'Singapore', 'Malaysia', 'Iraq', 'Saudi Arabia'],
            y = neg_speech
    )]

layout = go.Layout(
    title='<b>Countries with Greatest Number of Negative Speeches Delivered</b>',
    xaxis=dict(
        title='Country'
    ),
    yaxis=dict(
        title='Number of Negative Speeches'
    ),
    
)

fig = go.Figure(data=data, layout=layout)

plot(fig, filename='Countries with Most Negative Speeches Delivered.html')

'file:///Users/anushkocharyan/Desktop/UN_Sessions/Countries with Most Negative Speeches Delivered.html'